# Dokumentacja projektu NeuralPoker

NeuralPoker realizuje sieć neuronową do wyznaczania układu kart oraz prawdopodobieństwa wygranej z daną ręką i etapie rozdania (<i>preflop</i>, <i>flop</i>, <i>turn</i>, <i>river</i>) w pokerze Texas Hold'em. Projekt dostępny pod: https://github.com/kazimierczak-robert/NeuralPoker i kompatybilny z python3.

## Zewnętrzne biblioteki
W projekcie zawarte w folderze Libraries.

### Biblioteka deuces
Biblioteka wykorzystana do wyznaczenia układów kart oraz ich sił. Oryginalna postać znajduje się pod: https://github.com/worldveil/deuces i jest kompatybilna z python2.

### Biblioteka Monte-Carlo
Biblioteka wykorzystana do wyznaczenia prawdopodobieństw wygranej z daną ręką i etapie rozdania metodą Monte Carlo. Oryginalna postać znajduje się pod: https://github.com/Blind0ne/Monte-Carlo. Do działania wymagane jest pobranie biblioteki deuces.

## Generowanie zbioru treningowego i testowego
Krok wykorzystuje zewnętrze biblioteki opisane w nagłówku [Zewnętrzne biblioteki](## Zewnętrzne biblioteki). Do wygenerowania zbioru testowego i treningowego należy wykorzystać plik <code>SetGenerator.py</code>. Metoda <code>generate_sets</code> przyjmuje jako parametr liczbę wszystkich kart na danym etapie gry <code>all_cards_no</code> (2, 5, 6, 7). Liczebność zbiorów w zależności od parametru:

<table class="tg">
  <tr>
    <th class="tg-13j4">Parametr</th>
    <th class="tg-13j4">Zbiór testowy</th>
    <th class="tg-13j4">Zbiór treningowy</th>
  </tr>
  <tr>
    <td class="tg-yxcf">2</td>
    <td class="tg-yxcf">133<br></td>
    <td class="tg-yxcf">1195</td>
  </tr>
  <tr>
    <td class="tg-5q1d">5</td>
    <td class="tg-5q1d">3979</td>
    <td class="tg-5q1d">35803</td>
  </tr>
  <tr>
    <td class="tg-mg4r">6</td>
    <td class="tg-mg4r">3979</td>
    <td class="tg-mg4r">35803</td>
  </tr>
  <tr>
    <td class="tg-5q1d">7</td>
    <td class="tg-5q1d">3979</td>
    <td class="tg-5q1d">35803</td>
  </tr>
</table>

Metoda drukuje na standardowe wyjście numer wygenerowanej próbki (treningowej, testowej).

Część przykładowego zbioru treningowego dla parametru <code>all_cards_no=2</code>:
<code>
2,4,2,6,0.20840000000000003
1,6,4,13,0.23440000000000005
2,6,4,13,0.25
3,12,4,12,0.5028
1,6,3,6,0.24680000000000002
</code>
Karta reprezentowana parą liczb: kolor i figura. Dwie pary liczb to karty w ręce. Ostatnia liczba to prawdopodobieństwo wygranej.
Część przykładowego zbioru treningowego dla parametru <code>all_cards_no=6</code>:
<code>
1,5,4,5,1,12,3,7,2,13,4,1,8,0.12680000000000002
3,4,4,4,4,5,4,9,2,4,1,4,2,0.9996
1,6,4,6,1,8,2,8,2,1,2,2,7,0.28080000000000005
1,2,2,5,4,7,3,6,3,13,1,9,9,0.03520000000000001
4,4,3,7,4,6,3,9,4,10,1,7,8,0.11599999999999999
</code>
Karta reprezentowana parą liczb: kolor i figura. Ostatnie dwie liczby to układ (najsilniejszy) i prawdopodobieństwo wygranej. Dwie pierwsze pary to karty w ręce, pozostałe - na stole.

In [ ]:
from Sets.SetGenerator import SetGenerator
all_cards_no = 5
generator = SetGenerator()
generator.generate_sets(all_cards_no)

## Wczytywanie zbioru treningowego i testowego
Należy zaimportować pliki <code>SetImporter.py</code> i <code>SetGenerator.py</code>. Konstruktor klasy <code>SetImporter</code> jest dwuargumentowy: ścieżka do pliku ze zbiorem i liczba wszystkich kart na danym etapie gry.

In [1]:
from Sets.SetImporter import SetImporter
from Sets.SetGenerator import SetGenerator
all_cards_no = 5
train_set = SetImporter("{}/{}.data".format(SetGenerator.dir_path, "training", all_cards_no), all_cards_no)
test_set = SetImporter("{}/{}.data".format(SetGenerator.dir_path, "test", all_cards_no), all_cards_no)

## Model sieci neuronowej
W celu wytrenowania modelu i przetestowania go należy wykonać poniższy kod. Parametr metody <code>create</code> oznacza liczbę neuronów warstwy ukrytej.

In [ ]:
from Sets.SetImporter import SetImporter
from Sets.SetGenerator import SetGenerator
from Model.Model import Model

all_cards_no = 5
train_set = SetImporter("{}/poker-hand-{}-{}.data".format(SetGenerator.dir_path, "training", all_cards_no), all_cards_no)
test_set = SetImporter("{}/poker-hand-{}-{}.data".format(SetGenerator.dir_path, "test", all_cards_no), all_cards_no)

model = Model(train_set)
model.create(28)
model.train()
model.test(test_set)

Metoda <code>create</code> tworzy model sieci neuronowej z wykorzystaniem biblioteki <code>tensorflow</code>. Sieć składa się z warstwy wejściowej o 17 neuronach (13 figur + 4 kolory), jednej warstwy ukrytej <i>Dense</i>, która wykorzystuje funkcję aktywacji relu. W zależności od wybranego etapu gry (2 lub 5, 6, 7) sieć ma jedną (dla 2) lub 2 warstwy wyjściowe (dla 5, 6, 7). Warstwa wyjściowa z jednym neuronem wyznaczająca prawdopodobieństwo wygranej (regresja) wykorzystuje funkcję aktywacji sigmoid. Warstwa wyjściowa z 10 neuronami wyznaczająca układ kart (klasyfikacja) wykorzytuje funkcję aktywacji softmax. Wykorzystano metryki: acc (klasyfikacja), mse (regresja).

In [ ]:
def compile(self):
    if self.all_cards == 2:
        # metrics for regression
        self.model.compile(optimizer=tf.train.AdamOptimizer(), loss='mse', metrics=['mse'])
    else:
        # metrics for classification and regression
        self.model.compile(optimizer=tf.train.AdamOptimizer(),
                           loss={'output_layer_1': 'sparse_categorical_crossentropy', 'output_layer_2': 'mse'},
                           metrics={'output_layer_1': 'acc', 'output_layer_2': 'mse'})

def create(self, neurons_in_hidden_layer):
    if self.dataset is not None:
        self.model = keras.models.Sequential()
        # 17 = 13 ranks + 4 suits
        input_layer = keras.layers.Input((17*self.dataset.all_cards,), name="input_layer")
        hidden_layer = keras.layers.Dense(neurons_in_hidden_layer, activation=tf.nn.relu, name="hidden_layer")(input_layer)
        # 1 output in NN
        if self.dataset.all_cards == 2:
            output_layer = keras.layers.Dense(1, activation=tf.nn.sigmoid, name="output_layer")(hidden_layer)
            self.model = keras.models.Model(input_layer, output_layer)
        # 2 outputs in NN
        else:
            output_1 = keras.layers.Dense(10, activation=tf.nn.softmax, name="output_layer_1")(hidden_layer)
            output_2 = keras.layers.Dense(1, activation=tf.nn.sigmoid, name="output_layer_2")(hidden_layer)
            self.model = keras.models.Model(input_layer, [output_1, output_2])
        self.compile()

### Testowanie konfiguracji
Ciemniejszym kolorem oznaczono konfiguracje, dla których uzyskano najlepsze wyniki testów.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 12px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 12px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-13j4{font-weight:bold;font-size:12px;border-color:#efefef;text-align:center;vertical-align:top}
.tg .tg-yxcf{font-size:12px;background-color:#efefef;border-color:#efefef;text-align:center;vertical-align:top}
.tg .tg-5q1d{font-size:12px;border-color:#efefef;text-align:center;vertical-align:top}
.tg .tg-mg4r{font-size:12px;background-color:#efefef;color:#000000;border-color:#efefef;text-align:center;vertical-align:top}
.tg .tg-d99v{font-size:12px;background-color:#9b9b9b;color:#000000;border-color:#efefef;text-align:center;vertical-align:top}
</style>

* <code>all_cards_no</code> = 2

<table class="tg">
  <tr>
    <th class="tg-13j4">neurons_no</th>
    <th class="tg-13j4">loss</th>
    <th class="tg-13j4">mean_squared_error</th>
  </tr>
  <tr>
    <td class="tg-yxcf">8</td>
    <td class="tg-yxcf">0.0002728728868532926<br></td>
    <td class="tg-yxcf">0.0002728728868532926</td>
  </tr>
  <tr>
    <td class="tg-5q1d">16</td>
    <td class="tg-5q1d">0.00019329327915329486</td>
    <td class="tg-5q1d">0.00019329327915329486</td>
  </tr>
  <tr>
    <td class="tg-mg4r">24</td>
    <td class="tg-mg4r">0.0002716764865908772</td>
    <td class="tg-mg4r">0.0002716764865908772</td>
  </tr>
  <tr>
    <td class="tg-d99v">28</td>
    <td class="tg-d99v">0.00016631107428111136</td>
    <td class="tg-d99v">0.00016631107428111136</td>
  </tr>
  <tr>
    <td class="tg-mg4r">29</td>
    <td class="tg-mg4r">0.00021212051797192544</td>
    <td class="tg-mg4r">0.00021212051797192544</td>
  </tr>
  <tr>
    <td class="tg-5q1d">30</td>
    <td class="tg-5q1d">0.00021880857821088284</td>
    <td class="tg-5q1d">0.00021880857821088284</td>
  </tr>
  <tr>
    <td class="tg-mg4r">32</td>
    <td class="tg-mg4r">0.00019229181634727865</td>
    <td class="tg-mg4r">0.00019229181634727865</td>
  </tr>
</table>

* <code>all_cards_no</code> = 5

<table class="tg">
  <tr>
    <th class="tg-13j4">neurons_no</th>
    <th class="tg-13j4">loss</th>
    <th class="tg-13j4">output_layer_1_loss</th>
    <th class="tg-13j4">output_layer_2_loss</th>
    <th class="tg-13j4">output_layer_1_acc</th>
    <th class="tg-13j4">output_layer_2_mean_squared_error</th>
  </tr>
  <tr>
    <td class="tg-yxcf">8</td>
    <td class="tg-yxcf">0.8569035530090332</td>
    <td class="tg-yxcf">0.8307963013648987</td>
    <td class="tg-yxcf">0.026107240468263626</td>
    <td class="tg-yxcf">0.6337355375289917</td>
    <td class="tg-yxcf">0.026107240468263626</td>
  </tr>
  <tr>
    <td class="tg-5q1d">16</td>
    <td class="tg-5q1d">0.18359050154685974</td>
    <td class="tg-5q1d">0.1657908409833908</td>
    <td class="tg-5q1d">0.017799654975533485</td>
    <td class="tg-5q1d">0.9472096562385559</td>
    <td class="tg-5q1d">0.017799654975533485</td>
  </tr>
  <tr>
    <td class="tg-mg4r">24</td>
    <td class="tg-mg4r">0.23562563955783844</td>
    <td class="tg-mg4r">0.2202180176973343</td>
    <td class="tg-mg4r">0.015407627448439598</td>
    <td class="tg-mg4r">0.9678230285644531</td>
    <td class="tg-mg4r">0.015407627448439598</td>
  </tr>
  <tr>
    <td class="tg-d99v">28</td>
    <td class="tg-d99v">0.24058467149734497</td>
    <td class="tg-d99v">0.2283773422241211</td>
    <td class="tg-d99v">0.012207326479256153</td>
    <td class="tg-d99v">0.9690799117088318</td>
    <td class="tg-d99v">0.012207326479256153</td>
  </tr>
  <tr>
    <td class="tg-mg4r">29</td>
    <td class="tg-mg4r">0.29240182042121887</td>
    <td class="tg-mg4r">0.2786564528942108</td>
    <td class="tg-mg4r">0.01374536007642746</td>
    <td class="tg-mg4r">0.9658119678497314</td>
    <td class="tg-mg4r">0.01374536007642746</td>
  </tr>
  <tr>
    <td class="tg-5q1d">30</td>
    <td class="tg-5q1d">0.35394394397735596</td>
    <td class="tg-5q1d">0.34037789702415466</td>
    <td class="tg-5q1d">0.013566038571298122</td>
    <td class="tg-5q1d">0.9615384340286255</td>
    <td class="tg-5q1d">0.013566038571298122</td>
  </tr>
  <tr>
    <td class="tg-mg4r">32</td>
    <td class="tg-mg4r">0.39161327481269836</td>
    <td class="tg-mg4r">0.37983229756355286</td>
    <td class="tg-mg4r">0.011780978180468082</td>
    <td class="tg-mg4r">0.9610356688499451</td>
    <td class="tg-mg4r">0.011780978180468082</td>
  </tr>
</table>

* <code>all_cards_no</code> = 6

<table class="tg">
  <tr>
    <th class="tg-13j4">neurons_no</th>
    <th class="tg-13j4">loss</th>
    <th class="tg-13j4">output_layer_1_loss</th>
    <th class="tg-13j4">output_layer_2_loss</th>
    <th class="tg-13j4">output_layer_1_acc</th>
    <th class="tg-13j4">output_layer_2_mean_squared_error</th>
  </tr>
  <tr>
    <td class="tg-yxcf">8</td>
    <td class="tg-yxcf">1.1386266946792603</td>
    <td class="tg-yxcf">1.100320816040039</td>
    <td class="tg-yxcf">0.03830586373806</td>
    <td class="tg-yxcf">0.5389643311500549</td>
    <td class="tg-yxcf">0.03830586373806</td>
  </tr>
  <tr>
    <td class="tg-5q1d">16</td>
    <td class="tg-5q1d">0.5025573968887329</td>
    <td class="tg-5q1d">0.4704950153827667</td>
    <td class="tg-5q1d">0.0320623554289341</td>
    <td class="tg-5q1d">0.8627451062202454</td>
    <td class="tg-5q1d">0.0320623554289341</td>
  </tr>
  <tr>
    <td class="tg-mg4r">24</td>
    <td class="tg-mg4r">0.2888332009315491</td>
    <td class="tg-mg4r">0.2606068551540375</td>
    <td class="tg-mg4r">0.02822634018957615</td>
    <td class="tg-mg4r">0.9263448715209961</td>
    <td class="tg-mg4r">0.02822634018957615</td>
  </tr>
  <tr>
    <td class="tg-5q1d">28</td>
    <td class="tg-5q1d">0.33630427718162537</td>
    <td class="tg-5q1d">0.3104614317417145</td>
    <td class="tg-5q1d">0.025842837989330292</td>
    <td class="tg-5q1d">0.9281045794487</td>
    <td class="tg-5q1d">0.025842837989330292</td>
  </tr>
  <tr>
    <td class="tg-mg4r">29</td>
    <td class="tg-mg4r">0.3946927785873413</td>
    <td class="tg-mg4r">0.3673554062843323</td>
    <td class="tg-mg4r">0.02733737602829933</td>
    <td class="tg-mg4r">0.9263448715209961</td>
    <td class="tg-mg4r">0.02733737602829933</td>
  </tr>
  <tr>
    <td class="tg-d99v">30</td>
    <td class="tg-d99v">0.5774040818214417</td>
    <td class="tg-d99v">0.5508595705032349</td>
    <td class="tg-d99v">0.026544496417045593</td>
    <td class="tg-d99v">0.9301156401634216</td>
    <td class="tg-d99v">0.026544496417045593</td>
  </tr>
  <tr>
    <td class="tg-mg4r">32</td>
    <td class="tg-mg4r">0.44998517632484436</td>
    <td class="tg-mg4r">0.4231707453727722</td>
    <td class="tg-mg4r">0.02681444026529789</td>
    <td class="tg-mg4r">0.9147812724113464</td>
    <td class="tg-mg4r">0.02681444026529789</td>
  </tr>
</table>


* <code>all_cards_no</code> = 7
<table class="tg">
  <tr>
    <th class="tg-13j4">neurons_no</th>
    <th class="tg-13j4">loss</th>
    <th class="tg-13j4">output_layer_1_loss</th>
    <th class="tg-13j4">output_layer_2_loss</th>
    <th class="tg-13j4">output_layer_1_acc</th>
    <th class="tg-13j4">output_layer_2_mean_squared_error</th>
  </tr>
  <tr>
    <td class="tg-yxcf">8</td>
    <td class="tg-yxcf">1.3301163911819458</td>
    <td class="tg-yxcf">1.2595701217651367</td>
    <td class="tg-yxcf">0.0705462396144867</td>
    <td class="tg-yxcf">0.47561588883399963</td>
    <td class="tg-yxcf">0.0705462396144867</td>
  </tr>
  <tr>
    <td class="tg-5q1d">16</td>
    <td class="tg-5q1d">0.9917051196098328</td>
    <td class="tg-5q1d">0.928206741809845</td>
    <td class="tg-5q1d">0.063498355448246</td>
    <td class="tg-5q1d">0.6312217116355896</td>
    <td class="tg-5q1d">0.063498355448246</td>
  </tr>
  <tr>
    <td class="tg-mg4r">24</td>
    <td class="tg-mg4r">0.44883644580841064</td>
    <td class="tg-mg4r">0.39074623584747314</td>
    <td class="tg-mg4r">0.058090221136808395</td>
    <td class="tg-mg4r">0.8637506365776062</td>
    <td class="tg-mg4r">0.058090221136808395</td>
  </tr>
  <tr>
    <td class="tg-5q1d">28</td>
    <td class="tg-5q1d">0.6418846845626831</td>
    <td class="tg-5q1d">0.580848217010498</td>
    <td class="tg-5q1d">0.06103646755218506</td>
    <td class="tg-5q1d">0.8619909286499023</td>
    <td class="tg-5q1d">0.06103646755218506</td>
  </tr>
  <tr>
    <td class="tg-mg4r">29</td>
    <td class="tg-mg4r">0.6029195189476013</td>
    <td class="tg-mg4r">0.5420128703117371</td>
    <td class="tg-mg4r">0.06090662255883217</td>
    <td class="tg-mg4r">0.8501759767532349</td>
    <td class="tg-mg4r">0.06090662255883217</td>
  </tr>
  <tr>
    <td class="tg-d99v">30</td>
    <td class="tg-d99v">0.5111713409423828</td>
    <td class="tg-d99v">0.4530506730079651</td>
    <td class="tg-d99v">0.05812065303325653</td>
    <td class="tg-d99v">0.8866264224052429</td>
    <td class="tg-d99v">0.05812065303325653</td>
  </tr>
  <tr>
    <td class="tg-mg4r">32</td>
    <td class="tg-mg4r">0.571366548538208</td>
    <td class="tg-mg4r">0.5106329917907715</td>
    <td class="tg-mg4r">0.060733549296855927</td>
    <td class="tg-mg4r">0.8692810535430908</td>
    <td class="tg-mg4r">0.060733549296855927</td>
  </tr>
</table>

## Użytkowanie sieci neuronowej

### Zapis modelu do plików

In [ ]:
model.save("{}/model-{}".format(Model.dir_path, all_cards_no))

### Odczyt modelu z plików

In [ ]:
model.load("{}/model-{}".format(Model.dir_path, all_cards_no), all_cards_no)

### Predykcja wyników
Układy oznaczane są numerem. Karty reprezentowane są za pomocą konkatenacji znaków: figura i kolor.
<div>
<div style="margin-top:10px">
<table style="float:left; margin-right:40px" class="tg">
  <tr>
    <th class="tg-13j4">Numer układu<br></th>
    <th class="tg-13j4">Układ<br></th>
  </tr>
  <tr>
    <td class="tg-yxcf">0</td>
    <td class="tg-yxcf">Royal Straight Flush</td>
  </tr>
  <tr>
    <td class="tg-5q1d">1<br></td>
    <td class="tg-5q1d">Straight Flush</td>
  </tr>
  <tr>
    <td class="tg-mg4r">2</td>
    <td class="tg-mg4r">Four of a Kind</td>
  </tr>
  <tr>
    <td class="tg-5q1d">3</td>
    <td class="tg-5q1d">Full House</td>
  </tr>
  <tr>
    <td class="tg-yxcf">4</td>
    <td class="tg-yxcf">Flush</td>
  </tr>
  <tr>
    <td class="tg-5q1d">5</td>
    <td class="tg-5q1d">Straight</td>
  </tr>
  <tr>
    <td class="tg-mg4r">6</td>
    <td class="tg-mg4r">Three of a Kind</td>
  </tr>
  <tr>
    <td class="tg-5q1d">7</td>
    <td class="tg-5q1d">Two Pair</td>
  </tr>
  <tr>
    <td class="tg-yxcf">8</td>
    <td class="tg-yxcf">Pair</td>
  </tr>
  <tr>
    <td class="tg-5q1d">9</td>
    <td class="tg-5q1d">High Card</td>
  </tr>
</table>    
</div>
<div>
<table  style="float:left; margin-right:40px" class="tg">
  <tr>
    <th class="tg-13j4">Figura</th>
    <th class="tg-13j4">Znak</th>
  </tr>
  <tr>
    <td class="tg-yxcf">2,3,...,9</td>
    <td class="tg-yxcf">2,3,...,9</td>
  </tr>
  <tr>
    <td class="tg-5q1d">10<br></td>
    <td class="tg-5q1d">10</td>
  </tr>
  <tr>
    <td class="tg-mg4r">Jack</td>
    <td class="tg-mg4r">J</td>
  </tr>
  <tr>
    <td class="tg-5q1d">Queen</td>
    <td class="tg-5q1d">Q</td>
  </tr>
  <tr>
    <td class="tg-mg4r">King</td>
    <td class="tg-mg4r">K</td>
  </tr>
  <tr>
    <td class="tg-5q1d">Ace</td>
    <td class="tg-5q1d">A</td>
  </tr>
</table>
    </div>
<div>
<table class="tg" style="float:left">
  <tr>
    <th class="tg-13j4">Kolor</th>
    <th class="tg-13j4">Znak</th>
  </tr>
  <tr>
    <td class="tg-yxcf">Spades</td>
    <td class="tg-yxcf">s</td>
  </tr>
  <tr>
    <td class="tg-5q1d">Hearts</td>
    <td class="tg-5q1d">h</td>
  </tr>
  <tr>
    <td class="tg-yxcf">Diamonds</td>
    <td class="tg-yxcf">d</td>
  </tr>
  <tr>
    <td class="tg-5q1d">Clubs</td>
    <td class="tg-5q1d">c</td>
  </tr>
</table>
    </div>
</div>

Predykcja dla etapu gry z 2 kartami w ręce zwraca prawdopodobieństwo wygranej w rozdaniu. Dla innych etapów dodatkowo zwracany jest numer układu kart (najsilniejszego).

In [ ]:
print(model.predict(['5s', '5d', 'Td', 'Ts', 'Qs']))

### Przykładowy scenariusz użytkowania


In [6]:
from Model.Model import Model

all_cards_no = 5
model = Model()
model.load("{}/model-{}".format(Model.dir_path, all_cards_no), all_cards_no)
print(model.predict(['Qs', 'Qh', 'Qd', 'Kc', 'Kh']))

(3, 0.89730614)
